In [ ]:
import uproot
import awkward as ak

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import pypdftk

plt.rc("pdf", fonttype=42)
plt.rc("pdf", fonttype=42)
plt.rc("axes", labelsize=20)
plt.rc("xtick", labelsize=16, top=True, direction="in")
plt.rc("ytick", labelsize=16, right=True, direction="in")
plt.rc("axes", titlesize=20)
plt.rc("legend", fontsize=16)

import os
curdir = os.getcwd()
while not os.path.exists(".projectroot") and os.getcwd() != os.path.expanduser('~'):
    os.chdir("..")
try:
    proot = open(".projectroot").readline()
except:
    print("Could not find file '.projectroot'")
    print("Fallback: Look for the file 'rpad.py' in the current working directory (%s)" % curdir)
os.chdir(curdir)
import sys
if len(proot) > 0:
    sys.path.append(proot+"/python")
from rpad import rpad

os.makedirs("tmp", exist_ok=True)

dssds = ["U3", "U4", "U5"]
ids = [1, 2, 3]

In [ ]:
f = uproot.open(proot+"/data/deltaEcontained/all.root")
t = f["a"]
b1 = t.arrays(["E"], "id == 1")
b2 = t.arrays(["E"], "(id == 1) & (2 <= FI) & (FI <= 15) & (2 <= BI) & (BI <= 15)")
b3 = t.arrays(["E"], "id == 2")
b4 = t.arrays(["E"], "(id == 2) & (2 <= FI) & (FI <= 15) & (2 <= BI) & (BI <= 15)")

In [ ]:
regions = np.array([["beta", 450], ["hump", 770], ["peak1", 950], ["valley1", 1010], ["peak2", 1080], 
           ["valley2", 1180], ["peak3", 1290], ["valley3", 1330], ["peak4", 1380], 
           ["valley4", 1430], ["peak5", 1510], ["spurious", 1700], ["peak6", 1810], 
           ["valley6", 1880], ["peak7", 2010]])

plt.figure(figsize=(10, 7))
plt.hist(ak.flatten(b1.E), bins=np.arange(0, 3000, 5), log=True, histtype='step', label='U3 w/ rim')
plt.hist(ak.flatten(b2.E), bins=np.arange(0, 3000, 5), log=True, histtype='step', label='U3 w/o rim')
plt.hist(ak.flatten(b3.E), bins=np.arange(0, 3000, 5), log=True, histtype='step', label='U4 w rim')
plt.hist(1.005*ak.flatten(b4.E), bins=np.arange(0, 3000, 5), log=True, histtype='step', label='U4 w/o rim')
plt.grid()
plt.vlines(regions[:,1].astype(float), 0, 1e4, lw=1, color='k')
plt.legend()
plt.savefig("deltaEcontained-spec.pdf", bbox_inches='tight')
plt.show()

In [ ]:
fignames = []
for i in range(len(regions)):
    if i == 0:
        lower = 0.
    else:
        lower = float(regions[i - 1, 1])
    upper = float(regions[i, 1])
    name = regions[i, 0]
    energycut = "(%f <= E) & (E <= %f)" % (0, 500)
    b1 = t.arrays(["FI", "BI"], "(id == 1) & (2 <= FI) & (FI <= 15) & (2 <= BI) & (BI <= 15) & (%f <= E) & (E <= %f)" % (lower, upper))
    b2 = t.arrays(["FI", "BI"], "(id == 2) & (2 <= FI) & (FI <= 15) & (2 <= BI) & (BI <= 15) & (%f <= E) & (E <= %f)" % (lower, upper))
    x1, y1 = rpad(b1.FI, b1.BI)
    x2, y2 = rpad(b2.FI, b2.BI)
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    plt.sca(axes[0])
    plt.hist2d(x1 - 0.5, y1 - 0.5, bins=(np.arange(0.5, 17.5, 1), np.arange(0.5, 17.5, 1)), norm=mpl.colors.LogNorm())
    plt.gca().invert_yaxis()
    plt.xlabel('Front strip')
    plt.ylabel('Back strip')
    plt.xticks(range(2, 16+2, 2))
    plt.yticks(range(2, 16+2, 2))
    plt.title("U3 - region: %s" % name)
    if len(x1[~np.isnan(x1)]) > 0: 
        cbar = plt.colorbar()
    plt.sca(axes[1])
    plt.hist2d(x2 - 0.5, y2 - 0.5, bins=(np.arange(0.5, 17.5, 1), np.arange(0.5, 17.5, 1)), norm=mpl.colors.LogNorm())
    plt.gca().invert_yaxis()
    plt.xlabel('Front strip')
    plt.ylabel('Back strip')
    plt.xticks(range(2, 16+2, 2))
    plt.yticks(range(2, 16+2, 2))
    plt.title("U4 - region: %s" % name)
    if len(x2[~np.isnan(x2)]) > 0: 
        cbar = plt.colorbar()
    figname = 'tmp/%s.pdf' % name
    fignames.append(figname)
    plt.savefig(figname, bbox_inches='tight')
    plt.show()
pypdftk.concat(fignames, out_file='hitpattern.pdf')

In [ ]:
f.close()